In [1]:
import pandas as pd
import os
import numpy as np
from tqdm import tqdm
import warnings
import matplotlib.pyplot as plt

In [13]:
spread['direction'] = spread['0'].diff().apply(lambda x: 1 if x > 0 else -1 if x < 0 else 0)

In [24]:
def test_signal(spread, stock, future, name):
    test_data = spread
    spread_upper = spread['0'].quantile(0.8)
    spread_lower = spread['0'].quantile(0.2)
    test_data['signal'] = (test_data['0'] > spread_upper) | (test_data['0'] < spread_lower)
    test_data['direction'] = test_data['0'].diff().apply(lambda x: 1 if x > 0 else -1 if x < 0 else 0)
    indices = test_data.index[(test_data['signal'].shift() == False) & (test_data['signal'] == True)]
    signal_occur = test_data.loc[indices]
    test_stock = stock
    test_stock['direction'] = test_stock['price'].diff().apply(lambda x: 1 if x > 0 else -1 if x < 0 else 0)
    test_future = future
    test_future['direction'] = test_future['midQ'].diff().apply(lambda x: 1 if x > 0 else -1 if x < 0 else 0)
    stock_signal = test_stock.loc[indices]
    future_signal = test_future.loc[indices]
    stock_futures = pd.merge(future_signal, stock_signal, left_index=True, right_index=True, how='inner',suffixes=('_futures', '_stock')).dropna()
    signal_data = pd.merge(stock_futures, signal_occur, left_index=True, right_index=True, how='inner').dropna()
    together_diff = ((signal_data['direction'] == signal_data['direction_stock']) & (signal_data['direction'] * signal_data['direction_futures'] == -1)).sum()
    stock_lead = ((signal_data['direction'] == signal_data['direction_stock']) & (signal_data['direction'] * signal_data['direction_futures'] != -1)).sum()
    future_lead = ((signal_data['direction'] != signal_data['direction_stock']) & (signal_data['direction'] != signal_data['direction_futures'])).sum()
    total = len(indices)
    together_diff_ratio = together_diff / total
    stock_lead_larger_ratio = stock_lead/ total
    future_lead_ratio = future_lead / total
    summary = [together_diff_ratio,stock_lead_larger_ratio,future_lead_ratio]
    return summary
    #print(stock_lead,'\n')
    #print(stock_lead_larger ,'\n')
    #print(future_lead ,'\n')

In [25]:
Path_dir = "/Users/lyq/Documents/Github/IndependentProject/SummerIndependentProject_GP/project_data/Chris_data/"    
name_list = []
for i in os.listdir(Path_dir):
    name_list.append(i)
name_list.remove('.DS_Store')

In [26]:
Path = "/Users/lyq/Documents/Github/IndependentProject/SummerIndependentProject_GP/project_data/Chris_data/"
leading_result = []
for _ in tqdm(name_list):
    spread = pd.read_csv(Path + _ + '/spread' +'.csv.gz',index_col=0, parse_dates=True)
    stock = pd.read_csv(Path+ _ + '/stock' +'.csv.gz',index_col=0, parse_dates=True)
    future = pd.read_csv(Path+ _ + '/future' +'.csv.gz',index_col=0, parse_dates=True)
    result = test_signal(spread, stock, future, _)
    leading_result.append(result)

100%|███████████████████████████████████████████| 10/10 [05:47<00:00, 34.76s/it]


In [27]:
together_diff = [item[0] for item in leading_result]
stock = [item[1] for item in leading_result]
future = [item[2] for item in leading_result]
df = pd.DataFrame({'together':together_diff, 'stock' : stock,'future' : future}, index=name_list)

In [28]:
df

,together,stock,future
HCF-2498,0.031079,0.540609,0.428312
NAF-3105,0.027737,0.549572,0.422691
DBF-2610,0.011121,0.676834,0.312045
EHF-1319,0.035967,0.715201,0.248832
QXF-2615,0.018843,0.584472,0.396685
IIF-3006,0.025636,0.524893,0.449472
QWF-2388,0.034727,0.562509,0.402764
PEF-5425,0.041343,0.533929,0.424728
JBF-3443,0.048758,0.536704,0.414538
IPF-3035,0.016148,0.597867,0.385985
